In [45]:
import mlflow 

import pandas as pd
import numpy as np

from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error
from  sklearn.model_selection import train_test_split

In [46]:
df=pd.read_csv('matrice_avis_utilisateur.csv')
df

,movie,user,timestamp,rating
0,858,6040,956703932,4
1,2384,6040,956703954,4
2,593,6040,956703954,5
3,2019,6040,956703977,5
4,1961,6040,956703977,4
...,...,...,...,...
1000204,2399,4958,1046454338,1
1000205,1407,4958,1046454443,5
1000206,3264,4958,1046454548,4
1000207,2634,4958,1046454548,3


In [47]:
mlflow.autolog()

with mlflow.start_run():


    df_sparse = df.astype(pd.SparseDtype("float", 0))

    nmf = NMF(n_components=18)

    X_train, X_test = train_test_split(df_sparse, test_size=0.2, random_state=42)

    W = nmf.fit_transform(X_train.values)  # User matrix
    H = nmf.components_  # Item matrix

    # Appliquez la NMF sur l'ensemble de test (si nécessaire)
    nmf_result_test = nmf.transform(X_test.values)
   

    
    pred_train_matrix = np.dot(W,H)
    pred_test_matrix = np.dot(nmf_result_test,H)

    predicted_df_train = pd.DataFrame(pred_train_matrix, columns=df_sparse.columns)
    predicted_df_test = pd.DataFrame(pred_test_matrix, columns=df_sparse.columns)

    mse_train = mean_squared_error(pred_train_matrix,X_train.values)
    mse_test =  mean_squared_error(pred_test_matrix,X_test.values)
    
    mlflow.log_metric("mse train",mse_train )
    mlflow.log_metric("mse test",mse_test)


2024/03/04 01:06:52 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2024/03/04 01:06:56 WARNING mlflow.sklearn: Training metrics will not be recorded because training labels were not specified. To automatically record training metrics, provide training labels as inputs to the model training function.
2024/03/04 01:06:56 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


In [48]:
mse_train,mse_test


(31195.696851479835, 90978.14437944732)